This notebook is inspired by the work of CEM (Zarlenga et al. 2022) and CBM (Koh et al. 2020) papers. Please visit their GitHub repositories:
[CEM GitHub](https://github.com/mateoespinosa/cem) and [CBM GitHub](https://github.com/yewsiang/ConceptBottleneck).

# CBM: inspection - evaluating and manual inspection

There are four main steps:
1. Loading the dataset.
2. Initializing a CBM with InceptionV3 vision backbone for the dataset.
3. Load CBMs
4. Inspect model
5. Distribution plots
6. Confusion matrix (not coded)

## Step 1: Load Data

The first step is to load the data. The designed CBM class with the PyTorch Lightning Trainer takes in PyTorch DataLoader object.
Furthermore, it needs to contain three elements (in the following order):
1. the sample image, $\mathbf{x}$
2. the image label, $\mathbf{y}$
3. the concept labels, in binary format, $\mathbf{c}$

In [ ]:
#from cub_data_module import *
import logging
import torch
import numpy as np
import yaml
import torchvision.models as models
import pytorch_lightning as pl
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm import tqdm
from collections import Counter

In [ ]:
#import sys #for Kaggle
#sys.path.append("/kaggle/usr/lib/cub_data_module_confounded") #for Kaggle

import src.cub_data_module as cub_data_module

In [ ]:
def _update_config_with_dataset(
    config,
    train_dl,
    n_concepts,
    n_tasks,
    concept_map,
):
    config["n_concepts"] = config.get(
        "n_concepts",
        n_concepts,
    )
    config["n_tasks"] = config.get(
        "n_tasks",
        n_tasks,
    )
    config["concept_map"] = config.get(
        "concept_map",
        concept_map,
    )

    task_class_weights = None

    if config.get('use_task_class_weights', False):
        logging.info(
            f"Computing task class weights in the training dataset with "
            f"size {len(train_dl)}..."
        )
        attribute_count = np.zeros((max(n_tasks, 2),))
        samples_seen = 0
        for i, data in enumerate(train_dl):
            if len(data) == 2:
                (_, (y, _)) = data
            else:
                (_, y, _) = data
            if n_tasks > 1:
                y = torch.nn.functional.one_hot(
                    y,
                    num_classes=n_tasks,
                ).cpu().detach().numpy()
            else:
                y = torch.cat(
                    [torch.unsqueeze(1 - y, dim=-1), torch.unsqueeze(y, dim=-1)],
                    dim=-1,
                ).cpu().detach().numpy()
            attribute_count += np.sum(y, axis=0)
            samples_seen += y.shape[0]
        print("Class distribution is:", attribute_count / samples_seen)
        if n_tasks > 1:
            task_class_weights = samples_seen / attribute_count - 1
        else:
            task_class_weights = np.array(
                [attribute_count[0]/attribute_count[1]]
            )
    return task_class_weights

In [ ]:
def _generate_dataset_and_update_config(
    experiment_config
):
    if experiment_config.get("dataset_config", None) is None:
        raise ValueError(
            "A dataset_config must be provided for each experiment run!"
        )

    dataset_config = experiment_config['dataset_config']
    logging.debug(
        f"The dataset's root directory is {dataset_config.get('root_dir')}"
    )
    intervention_config = experiment_config.get('intervention_config', {})
    if dataset_config["dataset"] == "cub":
        data_module = cub_data_module
    else:
        raise ValueError(f"Unsupported dataset {dataset_config['dataset']}!")

    train_dl, val_dl, test_dl, imbalance, (n_concepts, n_tasks, concept_map) = \
        data_module.generate_data(
            config=dataset_config,
            seed=42,
            output_dataset_vars=True,
            root_dir=dataset_config.get('root_dir', None),
            model_inspection=True,
        )
    # For now, we assume that all concepts have the same
    # aquisition cost
    acquisition_costs = None
    if concept_map is not None:
        intervened_groups = list(
            range(
                0,
                len(concept_map) + 1,
                intervention_config.get('intervention_freq', 1),
            )
        )
    else:
        intervened_groups = list(
            range(
                0,
                n_concepts + 1,
                intervention_config.get('intervention_freq', 1),
            )
        )

    task_class_weights = _update_config_with_dataset(
        config=experiment_config,
        train_dl=train_dl,
        n_concepts=n_concepts,
        n_tasks=n_tasks,
        concept_map=concept_map,
    )
    return (
        train_dl,
        val_dl,
        test_dl,
        imbalance,
        concept_map,
        intervened_groups,
        task_class_weights,
        acquisition_costs,
    )

In [ ]:
yaml_path = "data/cub.yaml" # for local development, might need to use whole path.

with open(yaml_path, "r") as file:
    yaml_config = yaml.safe_load(file)
yaml_config["shared_params"]["dataset_config"]["root_dir"] = "/kaggle/input/cem-cub2000-filtered/" #for Kaggle, replace this with locally downloaded folder.
yaml_config["shared_params"]["dataset_config"]["num_workers"] = 4 #change depending on resources available.
yaml_config["shared_params"]["dataset_config"]["batch_size"] = 64 #change depending on resources available.

In [ ]:
train_dl, val_dl, test_dl, imbalance, concept_map, intervened_groups, task_class_weights, acquisition_costs = _generate_dataset_and_update_config(yaml_config["shared_params"])

## Step 2: Create the CBM
### Step 2.1 Define model for input to concepts
We first need to define a architecture that will extract the concepts from the input image.

For this, we used a pre-trained InceptionV3 model. We remove the last linear layer and make one that we can use for our task, so it is ready for fine-tuning.

In [ ]:
def latent_code_generator_model(output_dim=112):
    # Load pre-trained InceptionV3
    inception = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)

    # Remove auxiliary classifier (set to None)
    inception.aux_logits = False  # Disable aux_logits
    inception.AuxLogits = None  # Delete aux classifier branch

    inception.fc = torch.nn.Linear(2048, output_dim)  # Replace classification layer with output_dim

    return inception

### Step 2.2: define CBM model.
We need to define the following:
1. `n_concepts`: the number of concepts in the dataset (112).
2. `n_tasks`: the number of output labels in the dataset (200).
3. `concept_loss_weight`: the weight to use for the concept prediction loss during training of the CBM. Picked to be the same as the CEM paper.
4. `learning_rate` and `optimizer`: to use during training. Optimizer is Adam by default, otherwise SGD.
5. `c_extractor_arch`: the model architecture to use for going from the input space to the concepts.
6. `c2y_model` and `c2y_layers`: the model architecture to use for going from the concepts to the labels. It can be directly the model, like c_extractor_arch or the layers as a list. We choose to do a simple linear layer.

In [ ]:
from src.utils_cbm import *
from src.cbm import ConceptBottleneckModel

## Step 3: Load the CBMs

Now that we have both the dataset and the model defined, we can train our CEM
using Pytorch Lightning's wrappers for ease. This should be very simple via
Pytorch Lightning's `Trainer` once the data has been generated:

In [ ]:
def load_model(typ):
    cbm_model_new = ConceptBottleneckModel.load_from_checkpoint(
        checkpoint_path=f"models/{typ}.ckpt", #might need to be changed to local
        n_concepts=112,
        n_tasks=200,
        concept_loss_weight=yaml_config["shared_params"]["concept_loss_weight"],
        learning_rate=yaml_config["shared_params"]["learning_rate"],  # The learning rate to use during training.
        optimizer="sgd",
        c_extractor_arch=latent_code_generator_model, # Here we provide our generating function for the latent code generator model.
        c2y_model=None,
    )
    
    return cbm_model_new

In [ ]:
cbm_model_f = load_model(typ="cbm_buggy")
cbm_model_o = load_model(typ="cbm_oracle")

## Step 4: inspect the model

In [ ]:
# Before anything, however, let's get the underlying numpy arrays of our
# test dataset as they will be easier to work with
x_test, y_test, c_test = [], [], []
for (x, y, c) in tqdm(test_dl):
    x_test.append(x)
    y_test.append(y)
    c_test.append(c)
x_test = np.concatenate(x_test, axis=0)
y_test = np.concatenate(y_test, axis=0)
c_test = np.concatenate(c_test, axis=0)

In [ ]:
def evaluate_model(dataloader, cbm, x_test, y_test, c_test):
    #Now we are ready to generate the concept, label, and embedding predictions for
    #the test set using our trained CEM:

    # We will use a Trainer object to run inference in batches over our test
    # dataset
    trainer_inference = pl.Trainer(
        accelerator="gpu",
        devices="auto",
        logger=False, # No logs to be dumped for this trainer
    )
    batch_results = trainer_inference.predict(cbm, dataloader)
    
    # Then we combine all results into numpy arrays by joining over the batch
    # dimension
    c_pred = np.concatenate(
        list(map(lambda x: x[0].detach().cpu().numpy(), batch_results)),
        axis=0,
    )
    c_embs = np.concatenate(
        list(map(lambda x: x[1].detach().cpu().numpy(), batch_results)),
        axis=0,
    )
    # Reshape them so that we have embeddings (batch_size, k, emb_size)
    c_embs = np.reshape(c_embs, (c_test.shape[0], c_test.shape[1], -1))
    
    y_pred = np.concatenate(
        list(map(lambda x: x[2].detach().cpu().numpy(), batch_results)),
        axis=0,
    )

    ##And compute all the metrics of interest:
    # To match the dimensions of y_pred (5794, 200), and y_test (5794,):
    # We need to apply a softmax layer to get the class probabilities
    y_prob = softmax(y_pred, axis=1)
    
    # Then we get the highest probability for the classes
    y_pred_classes = np.argmax(y_prob, axis=1)  # Shape (5794,)



    ##########
    ## Compute test task accuracy
    ##########
    task_accuracy = accuracy_score(y_test, y_pred_classes)
    print(f"Our CBM's test task accuracy is {task_accuracy*100:.2f}%")

    ##########
    ## Compute test concept AUC
    ##########
    concept_auc = roc_auc_score(c_test, c_pred)
    print(f"Our CBM's test concept AUC is {concept_auc*100:.2f}%")

    return y_pred_classes, c_pred

In [ ]:
y_pred_classes, c_pred = evaluate_model(test_dl, cbm_model_o, x_test, y_test, c_test)

In [ ]:
def concepti(c_sem):
    # Get indices of True values
    indices = torch.nonzero((torch.tensor(c_sem) > 0.5).float(), as_tuple=True)
    #print(indices[1].tolist())

    #print(indices)

    concepts_str_list = []
    
    for concept in indices[0].tolist():
        selected_concept = cub_data_module.SELECTED_CONCEPTS[concept]
        concept_str = cub_data_module.CONCEPT_SEMANTICS[selected_concept]
        concepts_str_list.append(concept_str)
    return concepts_str_list

def plot_image2(x_org):
    # Display image
    #image_tensor = x_org.squeeze(0)# Remove the batch dimension
    
    #transform = transforms.ToPILImage()# Convert the tensor to a PIL image
    #image = transform(x_org)
    
    plt.imshow(x_org)
    plt.axis('off')
    plt.show()

In [ ]:
#Let's inspect the model by looking into 25 instances of the test set.

#Pick 10 random instances in the set
random_indices = np.random.choice(len(y_test), 10, replace=False)

for index in random_indices:
    plot_image2(np.transpose(x_test[index], (1, 2, 0)))
    print("y_p:", y_pred_classes[index])
    print("y_r:", y_test[index])
    print()

    print('c_p:')
    print("\n".join(concepti(c_pred[index])))
    #print('c_r':, c_test[i])

    print("="*80)

In [ ]:
y_pred_classes_f, c_pred_f = evaluate_model(test_dl, cbm_model_f, x_test, y_test, c_test)

In [ ]:
# For the same random matches, let's see what the buggy model predicts
for index in random_indices:
    plot_image2(np.transpose(x_test[index], (1, 2, 0)))
    print("y_p:", y_pred_classes_f[index])
    print("y_r:", y_test[index])
    print()

    print('c_p:')
    print("\n".join(concepti(c_pred_f[index])))
    #print('c_r':, c_test[i])

    print("="*80)

## Step 5: distribution plots
There is a strong suspicion that the buggy model is overclassifying a certain class. So this is investigation at this step.

In [ ]:
# Calculate the distributions
oracle_distribution = Counter(y_pred_classes)
real_distribution = Counter(y_test)
buggy_distribution = Counter(y_pred_classes_f)

# Create a list of all unique classes
all_classes = list(set(oracle_distribution.keys()).union(set(real_distribution.keys())).union(set(buggy_distribution.keys())))

# Create lists for the counts of each class in each distribution
oracle_counts = [oracle_distribution.get(cls, 0) for cls in all_classes]
real_counts = [real_distribution.get(cls, 0) for cls in all_classes]
buggy_counts = [buggy_distribution.get(cls, 0) for cls in all_classes]

# Plot the oracle distribution
plt.figure(figsize=(10, 5))
plt.bar(all_classes, oracle_counts, color='blue', label='Oracle distribution')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Oracle Distribution of Classes')
plt.legend()
plt.savefig('oracle_distribution.png')
plt.show()

# Plot the real distribution
plt.figure(figsize=(10, 5))
plt.bar(all_classes, real_counts, color='green', label='Real distribution')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Real Distribution of Classes')
plt.legend()
plt.savefig('real_distribution.png')
plt.show()

# Plot the buggy distribution
plt.figure(figsize=(10, 5))
plt.bar(all_classes, buggy_counts, color='red', label='Buggy distribution')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Buggy Distribution of Classes')
plt.legend()
plt.savefig('buggy_distribution.png')
plt.show()

## Step 6:
An alternative idea to the distribution plot would be to plot a confusion matrix. However, as there are 200 classes, it's very difficult to get them all plotted and the plots might become cluttered.